In [ ]:
# Cell 1: Setup and Imports
import os
import sys
import asyncio
import time

from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv


load_dotenv()


In [ ]:
# Cell 2: Initialize Groq API key and components
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
assert GROQ_API_KEY, "GROQ_API_KEY not set in environment variables."

# Load embeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [ ]:
# Cell 3: Load documents, split them, and build vector store
loader = WebBaseLoader("https://www.wikipedia.org/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_documents = text_splitter.split_documents(docs[:50])

vector_store = FAISS.from_documents(final_documents, embeddings)
retriever = vector_store.as_retriever()

final_documents


In [ ]:
# Cell 4: Define prompt and document chain
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_TOGETHER_API') 
llm = ChatOpenAI(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
    openai_api_key=OPENAI_API_KEY,
    openai_api_base="https://api.together.xyz/v1",
    temperature = 0
)

prompt = ChatPromptTemplate.from_template("""
Answer the question based on provided context only.
Please provide the best accurate response based on the question.
<context>
{context}
</context>
Question: {input}
""")

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
# Cell 5: User interaction loop

user_input = "Prompt Engineering"


start = time.process_time()
result = retrieval_chain.invoke({"input": user_input})
elapsed = time.process_time() - start

print("\n📥 Question:", user_input)
print("🧠 Answer:", result['answer'])
print(f"⏱️ Response Time: {elapsed:.2f} seconds\n")
